In [1]:
from optimizers.gradient import GradientOptimizer
from utils.general import CustomIgFold, FVbinder

/home/ubuntu/miniconda3/lib/python3.12/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [2]:
CIF = CustomIgFold()

Loading /home/ubuntu/miniconda3/lib/python3.12/site-packages/igfold/trained_models/IgFold/igfold_3.ckpt...
Loading /home/ubuntu/miniconda3/lib/python3.12/site-packages/igfold/trained_models/IgFold/igfold_5.ckpt...
Loading /home/ubuntu/miniconda3/lib/python3.12/site-packages/igfold/trained_models/IgFold/igfold_2.ckpt...
Loading /home/ubuntu/miniconda3/lib/python3.12/site-packages/igfold/trained_models/IgFold/igfold_1.ckpt...


In [3]:
light_sequence = "DIVLTQSPASQSASLGESVTITCLASQTIGTWLAWYQQKPGKSPQLLIYAATSLADGVPSRFSGSGSGTKFSFKISSLQAEDFVSYYCQQLYSTPYTFGGGTKLEIKR" #@param {type:"string"}
heavy_sequence = "EVKLQESGPSLVKPSQSLSLTCSVTGYSITSGYSWNWIRQFPGNKLEWMGYVNNDGLNNYNPSLKNRISITRDTSQNQFFLRFDSLTTEDTATYFCTLYGYDGAWFTYWGQGTLVTVSA" #@param {type:"string"}
target = FVbinder(vh=heavy_sequence, vl=light_sequence)
CIF.fold(target)

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.
/home/ubuntu/miniconda3/lib/python3.12/site-packages/igfold/utils/coordinates.py:18: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at ../aten/src/ATen/native/Cross.cpp:62.)
  n_vec = (b_coord - a_coord).expand(bc_vec.shape).cross(bc_vec)


In [4]:
humanized_vh = "EVQLQESGPGLVKPSETLSLTCAVSGYSITSGYSWNWIRQPPGNKLEWMGYVNNDGLNNYNPSLKSRITITRDTSKNQFSLKFSSVTAADTAVYFCTLYGYDGAWFTYWGQGTLVTVSS"
humanized_vl = "DIVLTQSPSSLSASVGDRVTITCLASQTIGTWLAWYQQKPGKSPQLLIYAATSLADGVPSRFSGSGSGTKFTLTISSLQPEDFATYYCQQLYSTPYTFGGGTKVEIKR"
humanized = FVbinder(vh=humanized_vh, vl=humanized_vl)

In [5]:
optimizer = GradientOptimizer(lr=0.05, sub_infinity=10, start=target, target=target, verbose=True, tensorboard=True)

In [6]:
optimizer.step(500)

  0%|          | 0/500 [00:00<?, ?it/s]

IndexError: tensors used as indices must be long, int, byte or bool tensors

In [16]:
optimizer.generator.logits[0][1]

tensor([   -inf,    -inf,    -inf,    -inf,    -inf, -4.1281, -4.0444, 12.0845,
         5.8196, -4.1159, -4.0830, -1.9722, -4.0156, -4.0865, -4.0878, -4.0759,
        -4.1504, -4.0485, -4.0195, -4.0683, -4.1643, -4.0628, -4.0996, -4.0658,
        -4.1590], device='cuda:0', grad_fn=<SelectBackward0>)

In [17]:
print(optimizer.best_valid_binder)

FVbinder(
vh='DVQLQESGPSLVKPSHSLSLTCSVTGYSITSGYHWNWIRQFPGNKLEWMGYHNNDGLNNYNPSLKNRISITRDTSKNQFFLRFHSVTTEDTATYFCTHYGYDGAWFTYWGQGTLVTVSA',
vl='DIHMTQSPASQSASLGERVTITCLASQTIGTWLAWYHQKPGKAPNLLIYAATSLADGVPSRFSGSGSGTNFSFNISSLQAEDFVSYYCQQLYSTPYTFGNGTKLEIK'
)


In [20]:
target.vh.align(optimizer.best_valid_binder.vh, humanized.vh)

EVKLQESGPSLVKPSQSLSLTCSVTGYSITSGYSWNWIRQFPGNKLEWMGYVNNDGLNNYNPSLKNRISITRDTSQNQFFLRFDSLTTEDTATYFCTLYGYDGAWFTYWGQGTLVTVSA
+|+||||||||||||.|||||||||||||||||.|||||||||||||||||.|||||||||||||||||||||||+|||||||.|+|||||||||||.|||||||||||||||||||||
DVQLQESGPSLVKPSHSLSLTCSVTGYSITSGYHWNWIRQFPGNKLEWMGYHNNDGLNNYNPSLKNRISITRDTSKNQFFLRFHSVTTEDTATYFCTHYGYDGAWFTYWGQGTLVTVSA
+||||||||.|||||.+|||||+|+||||||||.||||||.||||||||||.|||||||||||||+||+||||||||||.|+|.|||..|||.||||.||||||||||||||||||||+
EVQLQESGPGLVKPSETLSLTCAVSGYSITSGYSWNWIRQPPGNKLEWMGYVNNDGLNNYNPSLKSRITITRDTSKNQFSLKFSSVTAADTAVYFCTLYGYDGAWFTYWGQGTLVTVSS
                         ^^^^^^^^                   ^^^^^                                         ^^^^^^^^^^           

In [21]:
target.vl.align(optimizer.best_valid_binder.vl, humanized.vl)

DIVLTQSPASQSASLGESVTITCLASQTIGTWLAWYQQKPGKSPQLLIYAATSLADGVPSRFSGSGSGTKFSFKISSLQAEDFVSYYCQQLYSTPYTFGGGTKLEIK
||.+|||||||||||||.||||||||||||||||||.|||||+|.||||||||||||||||||||||||.|||.|||||||||||||||||||||||||.|||||||
DIHMTQSPASQSASLGERVTITCLASQTIGTWLAWYHQKPGKAPNLLIYAATSLADGVPSRFSGSGSGTNFSFNISSLQAEDFVSYYCQQLYSTPYTFGNGTKLEIK
||.+||||+|.|||+|+|||||||||||||||||||.|||||+|.||||||||||||||||||||||||.|+..|||||.|||.+||||||||||||||.|||+|||
DIVLTQSPSSLSASVGDRVTITCLASQTIGTWLAWYQQKPGKSPQLLIYAATSLADGVPSRFSGSGSGTKFTLTISSLQPEDFATYYCQQLYSTPYTFGGGTKVEIK
                       ^^^^^^^^^^^               ^^^^^^^                                ^^^^^^^^^          

In [ ]:
humanized.vh.align(optimizer.best_valid_binder.vh)

In [18]:
humanized.vh.align(optimizer.best_valid_binder.vh)

EVQLQESGPGLVKPSETLSLTCAVSGYSITSGYSWNWIRQPPGNKLEWMGYVNNDGLNNYNPSLKSRITITRDTSKNQFSLKFSSVTAADTAVYFCTLYGYDGAWFTYWGQGTLVTVSS
+||||||||.|||||.+|||||+|+||||||||.||||||.||||||||||.|||||||||||||+||+||||||||||.|+|.|||..|||.||||.||||||||||||||||||||+
DVQLQESGPSLVKPSHSLSLTCSVTGYSITSGYHWNWIRQFPGNKLEWMGYHNNDGLNNYNPSLKNRISITRDTSKNQFFLRFHSVTTEDTATYFCTHYGYDGAWFTYWGQGTLVTVSA
                         ^^^^^^^^                   ^^^^^                                         ^^^^^^^^^^           

In [19]:
humanized.vl.align(optimizer.best_valid_binder.vl)

DIVLTQSPSSLSASVGDRVTITCLASQTIGTWLAWYQQKPGKSPQLLIYAATSLADGVPSRFSGSGSGTKFTLTISSLQPEDFATYYCQQLYSTPYTFGGGTKVEIK
||.+||||+|.|||+|+|||||||||||||||||||.|||||+|.||||||||||||||||||||||||.|+..|||||.|||.+||||||||||||||.|||+|||
DIHMTQSPASQSASLGERVTITCLASQTIGTWLAWYHQKPGKAPNLLIYAATSLADGVPSRFSGSGSGTNFSFNISSLQAEDFVSYYCQQLYSTPYTFGNGTKLEIK
                       ^^^^^^^^^^^               ^^^^^^^                                ^^^^^^^^^          

In [10]:
from abnumber import Chain

In [11]:
seq = "ERKLQESGPSLMKPSQSLSLTCSVTGYSITSGYSWNWVRQFPGNGPEWMQNVNNDGLNNYNPSLKCRISITVITSLNQFFLRFDSLTTEDTATYECTDYGYDGAWFTYWGQGTLKACSK"

In [12]:
chain = Chain(seq, scheme="chothia")

In [13]:
a.v_gene

NameError: name 'a' is not defined

In [ ]:
optimizer.best_valid_binder.vh.renumber("imgt").find_human_germlines

In [ ]:
def v_gene_identity(sequence):
    c = Chain(sequence, scheme="imgt")
    v_germline_chains, _ = c.find_human_germlines(limit=1)
    top_v = v_germline_chains[0]

    num_v_matches = sum(top_v.positions.get(pos) == aa for pos, aa in c)

    v_identity = num_v_matches / len(top_v)
    return v_identity

In [ ]:
v_gene_identity(optimizer.best_valid_binder.vh.seq)

0.5773195876288659

In [ ]:
v_gene_identity(optimizer.best_valid_binder.vl.seq)

0.49473684210526314

In [ ]:
chain = Chain(seq, 'imgt')

v_germline_chains, j_germline_chains = chain.find_human_germlines(limit=1)
top_v, top_j = v_germline_chains[0], j_germline_chains[0]

num_v_matches = sum(top_v.positions.get(pos) == aa for pos, aa in chain)
num_j_matches = sum(top_j.positions.get(pos) == aa for pos, aa in chain)

v_identity = num_v_matches / len(top_v)
j_identity = num_j_matches / len(top_j)
full_identity = (num_v_matches + num_j_matches) / len(chain)

In [ ]:
v_identity

0.5773195876288659